In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("."))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.io.json import json_normalize
from sklearn.ensemble import RandomForestRegressor as RFF

from pprint import pprint

plt.rcParams["figure.figsize"] = (15,6)

In [ ]:
import sys
sys.path.append('..')

from fastai.structured import *

In [ ]:
%%time

json_cols = ['device', 'geoNetwork', 'totals', 'trafficSource']

json_conv = {col: json.loads for col in (json_cols)}

df_train = pd.read_csv("../input/train.csv", 
    dtype={'fullVisitorId': str},
    converters=json_conv,
    low_memory=True,
    parse_dates=['date'])


In [ ]:
add_datepart(df_train,'date')


In [ ]:
# convert json columns into signle columns
def decode_json(df_raw,json_cols):
    df = df_raw.copy()
    for column in json_cols:
        tdf = json_normalize(df[column])
        df = df.drop(columns = [column])
        tdf.columns = [column + '_' + col for col in tdf.columns]
        df = df.merge(tdf, left_index=True, right_index=True)
    return df



In [ ]:
import math
from  pprint import pprint
def rmse(x,y): 
    return math.sqrt(((x-y)**2).mean())
def split_vals(a,n) : return a[:n],a[n:]
def print_score(m):
    res = {'rmse_train':rmse(m.predict(X_train), y_train), 'rmse_valid':rmse(m.predict(X_valid), y_valid),
                'r2_train':m.score(X_train, y_train), 'r2_valid':m.score(X_valid, y_valid)}
    if hasattr(m, 'oob_score_'): res['r2_oob']= m.oob_score_
    pprint(res)

In [ ]:
%%time
df_train = decode_json(df_train,json_cols)


In [ ]:
to_keep_train = ['totals_pageviews', 'sessionId', 'totals_hits', 'visitStartTime', 'visitId', 'geoNetwork_country',
        'visitNumber', 'geoNetwork_networkDomain', 'geoNetwork_city', 'geoNetwork_region',
       'Day', 'totals_newVisits', 'device_operatingSystem', 'Dayofweek', 'geoNetwork_metro', 'Dayofyear',
       'trafficSource_source', 'Week', 'trafficSource_keyword', 'channelGrouping', 'device_browser', 
       'trafficSource_medium', 'device_isMobile',
       'totals_transactionRevenue']


to_keep_test = ['totals_pageviews', 'sessionId', 'totals_hits', 'visitStartTime', 'visitId', 'geoNetwork_country',
       'visitNumber', 'geoNetwork_networkDomain', 'geoNetwork_city', 'geoNetwork_region',
       'Day', 'totals_newVisits', 'device_operatingSystem', 'Dayofweek', 'geoNetwork_metro', 'Dayofyear',
       'trafficSource_source', 'Week', 'trafficSource_keyword', 'channelGrouping', 'device_browser',
       'trafficSource_medium', 'device_isMobile']
# something wrong with trafficSource_isTrueDirect in test set




In [ ]:
df_train = df_train[to_keep_train]


In [ ]:
cols_to_replace = {
    'socialEngagementType' : 'Not Socially Engaged',
    'device_browserSize' : 'not available in demo dataset', 
    'device_flashVersion' : 'not available in demo dataset', 
    'device_browserVersion' : 'not available in demo dataset', 
    'device_language' : 'not available in demo dataset',
    'device_mobileDeviceBranding' : 'not available in demo dataset',
    'device_mobileDeviceInfo' : 'not available in demo dataset',
    'device_mobileDeviceMarketingName' : 'not available in demo dataset',
    'device_mobileDeviceModel' : 'not available in demo dataset',
    'device_mobileInputSelector' : 'not available in demo dataset',
    'device_operatingSystemVersion' : 'not available in demo dataset',
    'device_screenColors' : 'not available in demo dataset',
    'device_screenResolution' : 'not available in demo dataset',
    'geoNetwork_city' : 'not available in demo dataset',
    'geoNetwork_cityId' : 'not available in demo dataset',
    'geoNetwork_latitude' : 'not available in demo dataset',
    'geoNetwork_longitude' : 'not available in demo dataset',
    'geoNetwork_metro' : ['not available in demo dataset', '(not set)'], 
    'geoNetwork_networkDomain' : 'unknown.unknown', 
    'geoNetwork_networkLocation' : 'not available in demo dataset',
    'geoNetwork_region' : 'not available in demo dataset',
    'trafficSource_adwordsClickInfo.criteriaParameters' : 'not available in demo dataset',
    'trafficSource_campaign' : '(not set)', 
    'trafficSource_keyword' : '(not provided)'
}

In [ ]:
%%time
df_train = df_train.replace(cols_to_replace, 'NA')

In [ ]:
df_train.totals_transactionRevenue = df_train['totals_transactionRevenue'].fillna(0).astype(float)

In [ ]:
train_cats(df_train)

In [ ]:
to_sort = ['totals_pageviews','totals_hits']
for cat in to_sort:
    orders = np.array(sorted(df_train[cat].cat.categories.astype(np.int32))).astype(np.str)
    df_train[cat].cat.reorder_categories(orders,inplace=True)

In [ ]:
df_trn ,y_trn, nas = proc_df(df_train,'totals_transactionRevenue')

In [ ]:
y_trn = np.log1p(y_trn)

In [ ]:
n_trn = 850000
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)

In [ ]:
%%time
m = RFF(n_estimators=160, max_features=0.5, n_jobs=-1, oob_score=True,min_samples_leaf=13)
m.fit(X_train,y_train)


In [ ]:
print_score(m)

In [ ]:
%%time

json_cols = ['device', 'geoNetwork', 'totals', 'trafficSource']

json_conv = {col: json.loads for col in (json_cols)}


df_test = pd.read_csv('../input/test.csv',
    dtype={'fullVisitorId': str},
    converters=json_conv,
    parse_dates=['date'])



In [ ]:
fullVisitorId = df_test.fullVisitorId

In [ ]:

add_datepart(df_test,'date')

df_test = decode_json(df_test,json_cols)

df_test = df_test[to_keep_test]

df_test = df_test.replace(cols_to_replace, 'NA')



In [ ]:
apply_cats(df_test,df_train)

In [ ]:
X_test,_ ,_ = proc_df(df_test)

In [ ]:
to_sort = ['totals_pageviews','totals_hits']
for cat in to_sort:
    orders = np.array(sorted(df_test[cat].cat.categories.astype(np.int32))).astype(np.str)
    df_test[cat].cat.reorder_categories(orders,inplace=True)

In [ ]:
submit = m.predict(X_test)

In [ ]:
my_submission = pd.DataFrame({'fullVisitorId':fullVisitorId,'PredictedLogRevenue':submit})
my_submission = my_submission.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()

In [ ]:
my_submission.to_csv('submission.csv', index=False)


In [ ]:
my_submission.head()